## add_conditional_edges 를 사용해서 tool을 이용하는 방법

In [1]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolNode, tools_condition

tool = TavilySearchResults(max_results=2)
tools = [tool]
tool_node = ToolNode(tools)

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    result = llm_with_tools.invoke(state["messages"])
    return {"messages": [result]}

In [4]:
from langgraph.graph import StateGraph

graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)

graph_builder.set_entry_point("chatbot")
graph = graph_builder.compile()

In [5]:
graph.invoke({"messages": {"role": "user", "content": "지금 한국 대통령은 누구야?"}})

{'messages': [HumanMessage(content='지금 한국 대통령은 누구야?', additional_kwargs={}, response_metadata={}, id='e3dc1259-5018-46c6-a38e-0afd8f427091'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_r0JX9ccAFcLkoSNt4eExIESd', 'function': {'arguments': '{"query":"현재 한국 대통령"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 87, 'total_tokens': 107, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-9b2c354e-260d-44a4-a4ab-e77316f0fa8d-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '현재 한국 대통령'}, 'id': 'call_r0JX9ccAFcLkoSNt4eExIESd', 'type': 'tool_call'}],